In [1]:
from fewshot.data_provider.dataset import Dataset
from fewshot.backbones import ConvNet

from fewshot.algorithms.backbone_pretrain import build_one_layer_classifier, cross_entropy_train
from fewshot.algorithms.fewshot_models import BaselineFewShotModel

import numpy as np
import tqdm
import scipy.stats as st

from IPython.display import clear_output

### Init

Set up all input data and params

In [3]:
# common
seed = 11

img_width = 84
img_height = 84
img_depth = 3

# classes
base_num_classes = 64
val_num_classes = 16  # not used in baseline algo
novel_num_classes = 20

# backbone training
backbone_training_batch_size = 32
backbone_training_epochs = 10

# fewshot training and testing
n_way = 5
k_shot = 5
training_batch_size = 4
training_batches_per_episode = 25
query_samples_per_class = 16

n_episodes = 50

In [4]:
dataset_dir = "../../../data/mini_imagenet/"
dataset = Dataset(dataset_dir=dataset_dir, csv_name="data_84x84.csv", image_size=(img_width, img_height))

### Backbone
Create backbone dataset (80 classes), split to train and validattion

In [5]:
backbone_dataset, val_fewshot_dataset = dataset.split_by_classes(train_size=base_num_classes,
                                                                 random_state=seed)

Split by classes with train size = 64 (seed = 11)
Train classes: 64
Test classes: 36
Train data: 38400 samples
Test data:  21600 samples


/mnt/disk/venv/dpy3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
val_dataset, fewshot_dataset = val_fewshot_dataset.split_by_classes(train_size=val_num_classes,
                                                                    random_state=seed)

Split by classes with train size = 16 (seed = 11)
Train classes: 16
Test classes: 20
Train data: 9600 samples
Test data:  12000 samples


Create classifier

In [7]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))
backbone_classifier = build_one_layer_classifier(backbone, backbone_dataset.n_classes)

Instructions for updating:
Colocations handled automatically by placer.


Load weights if we have presaved

Train

In [8]:
backbone.set_trainable(True)

In [9]:
cross_entropy_train(
    backbone_classifier,
    backbone_dataset.get_batch_generator(batch_size=backbone_training_batch_size, shuffle=True),
    n_epochs=backbone_training_epochs
)

Epoch 1/10
1200/1200 [==============================] - 204s 170ms/step - loss: 3.0932 - acc: 0.2360
Epoch 2/10
1200/1200 [==============================] - 201s 168ms/step - loss: 2.3561 - acc: 0.3841
Epoch 3/10
1200/1200 [==============================] - 202s 168ms/step - loss: 2.0388 - acc: 0.4545
Epoch 4/10
1200/1200 [==============================] - 201s 167ms/step - loss: 1.8135 - acc: 0.5084
Epoch 5/10
1200/1200 [==============================] - 200s 167ms/step - loss: 1.6285 - acc: 0.5525
Epoch 6/10
1200/1200 [==============================] - 198s 165ms/step - loss: 1.4645 - acc: 0.5912
Epoch 7/10
1200/1200 [==============================] - 198s 165ms/step - loss: 1.3124 - acc: 0.6297
Epoch 8/10
1200/1200 [==============================] - 197s 165ms/step - loss: 1.1749 - acc: 0.6649
Epoch 9/10
1200/1200 [==============================] - 198s 165ms/step - loss: 1.0520 - acc: 0.6947
Epoch 10/10
1200/1200 [==============================] - 198s 165ms/step - loss: 0.9318 - a

Save weights if we want in future to continue to train backbone 

### Few Shot
Train fewshot model

In [10]:
accs = []
for episode in tqdm.tqdm_notebook(range(n_episodes)):
    fewshot_model = BaselineFewShotModel(backbone, n_way)
    
    # not very simple way to prepare eposide support and query sets
    # FIXME: in future replace it with @bobbythehiver .few_shot_episode_generator() implementation
    # and @schoooler tester version
    episode_dataset, _ = fewshot_dataset.split_by_classes(train_size=n_way, random_state=episode)
    
    # subset support and query datasets
    episode_support, episode_left = episode_dataset.split_by_objects(train_size=k_shot, random_state=episode)
    episode_query, _ = episode_left.split_by_objects(train_size=query_samples_per_class, random_state=episode)
    
    
    fewshot_model.fit(episode_support.get_batch_generator(batch_size=training_batch_size, shuffle=True),
                      n_epochs=training_batches_per_episode)
    
    # calc accuracy
    out = fewshot_model.predict(episode_query.get_batch_generator(batch_size=training_batch_size, shuffle=False))
    classes = np.array(episode_query.classes)
    acc = np.mean(classes[np.argmax(out, axis=1)] == episode_query.dataframe["class"].values)
    
    accs.append(acc)
    clear_output()
    
    print("Mean accuracy (by {} episodes): {:.2f}%".format(episode + 1, np.mean(accs) * 100))

Mean accuracy (by 50 episodes): 54.35%



### Report

Calc 95% confidence interval and report

In [11]:
acc_int = st.t.interval(0.95, len(accs) - 1, loc=np.mean(accs), scale=st.sem(accs))

In [12]:
print("5-shot test metric: {}% +- {}%".format(round(np.mean(accs) * 100, 2),
                                              round((np.mean(acc_int) - acc_int[0]) * 100, 2)))

5-shot test metric: 54.35% +- 2.6%
